In [48]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import z2_save_jaspar, z1_save_oligos
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [49]:
#LOAD JASPAR & OLIGO DATA
jaspar = z2_save_jaspar.load_jaspar()

In [50]:

oligos, oligos_by_exp_some,oligos_by_exp = z1_save_oligos.load_oligos()
oligos_by_exp = oligos_by_exp.filter(regex='^(?!exp|\\.).*')
oligos_by_exp["exp_nm"] = oligos_by_exp.index.get_level_values("exp").to_series().apply(lambda x:re.compile('(.*)_BR').search(x).groups()[0]).values
oligos_by_exp = oligos_by_exp.reset_index()


oligos_by_exp["exp_ct"] = oligos_by_exp.exp_nm.apply(lambda x:re.compile("(U2OS|DLD1|HCT116)").search(x).groups()[0])
oligos_by_exp["exp_type"] = oligos_by_exp.exp_nm.apply(lambda x:"U2OS_NFKB" if "NFKB" in x 
                                                     else ("HCT116_GEM") if "Gem" in x
                                                     else re.compile("(U2OS|DLD1|HCT116)").search(x).groups()[0]+"_WT")

ranks = oligos_by_exp.loc[lambda x:x.mutant_num == 0].groupby("exp_type").\
    apply(lambda x:x.groupby("oligo").mu.mean().sort_values(ascending=False).\
          reset_index().reset_index().set_index("oligo").\
        rename({"index":"ranksort"},axis="columns").\
        ranksort).unstack(level=0).rename(lambda x: x+"_rank",axis = "columns")
oligos_by_exp = oligos_by_exp.join(ranks, on="oligo")

obe = oligos_by_exp

#LOADS MOTIF DATA
motif_oligos_data = pd.read_csv("../out/0722_motif_oligos_data.csv")

motif_oligos_data =  motif_oligos_data.join(jaspar[["threshold_patser"]],on="jaspar_id")
motif_oligos_data["affine_score"] = motif_oligos_data.score - motif_oligos_data.threshold_patser

KeyboardInterrupt: 

In [ ]:

ranks_by_rep = oligos_by_exp.loc[lambda x:x.mutant_num == 0].groupby(["exp_type","rep"]).\
    apply(lambda x:x.groupby("oligo").mu.mean().sort_values(ascending=False).\
          reset_index().reset_index().set_index("oligo").\
        rename({"index":"ranksort"},axis="columns").\
        ranksort).unstack(level=0).rename(lambda x: x+"_rank",axis = "columns")

In [ ]:
obe_w_repchanges = oligos_by_exp.join(
    (ranks_by_rep.loc[1] - ranks_by_rep.loc[2]).rename(lambda x:x+"_repchange",axis ="columns")
    ,on="oligo")




In [ ]:
intervals = pd.read_csv("../out/0729_intervals.csv")

In [ ]:
affine_score_threshold = 1

### IDENTIFICATION AND DESCRIPTION OF HIGHLY EXPRESSED ENHANCER CANDIDATES IN MULTIPLE CELL TYPES ###

####  Where are the promoters/enhancers (STARR-Seq peaks) as defined by this assay for this data in an overall pool? ####  
In the aggregate pooled data, on average [n_tx_mean] transcripts are identified for [n_bcs_total] barcodes (n_bcs_per_oligos. The number of transcripts can be counted per barcode, giving a number for the expression rate of the that barcode which is comparable between [SEE APPENDIX A, TRANSCRIPT COUNTING] barcoded plasmids in all of our pooled sequencing. We call this transcription rate “μ” the pooled expression of a given oligo, which can be defined on a per-experiment or pooled basis.

In [ ]:
pprint({
    "n_tx_mean":oligos.n_transcripts.mean(),
    "n_bcs_total":oligos.n_bcs.sum(),
    "n_bcs_avg":oligos.n_bcs.mean(),
})

Considering μ over all oligo start positions, we observe ~4-5 variants of each oligo and average this quantity together to define an average expression level. Considering all such oligo-averaged expression levels, we can define peaks of expression, or putative enhancer sequences. Going forward, we define the top 5% of oligo positions by location as potential enhancer sequences “POOL-activators”. A total of [POOL_activator_len] positions of 2000 are captured by this method and defined as pooled enhancers. Likewise, the bottom 5% “POOL-repressors” of oligos by expression are split into a separate group and identified as repressors, whereas the bottom 50%of oligos are identified and described as the null background “POOL-null”.

In [ ]:
wt_oligos = oligos.loc[lambda x: x.mutant_num ==0]
POOL_activators = wt_oligos.loc[lambda x: x.mu > x.mu.quantile(.95)]
POOL_repressors = wt_oligos.loc[lambda x: x.mu < x.mu.quantile(.05)]
POOL_null = wt_oligos.loc[lambda x: x.mu < x.mu.quantile(.5)]

print({
    "POOL_activator_len": len(POOL_activators),
    "POOL_repressor_len": len(POOL_repressors),
    "POOL_null_len": len(POOL_null),
})

Second performing the same analysis as above, we identified a set of 5% expressed oligos, repressed oligos, and null / negative control oligos for each replicate of each experiment “{EXP_N}-activators” / “{EXP_N}-repressors”. For each experiment, we then take an intersection of the activators, repressors, and null negative control oligos observed in the two replicates of each experiment, finding that on average 67% of these oligos are observed in both replicates over all experiments. Looking at each experiment individually, the U2OS and DLD1 cell types have replicable discovery rates of 79% and 67% respectively, with U2OS cell-type having the highest replicability overall We find that activators are overall more replicable than repressors, which is as expected based upon the distribution of log scores for each. 

In [ ]:
EXP_REP_activators = oligos_by_exp.loc[lambda x:x.mutant_num==0].groupby("exp").apply(lambda x: x.loc[x.mu > x.mu.quantile(.95)])
EXP_REP_repressors = oligos_by_exp.loc[lambda x:x.mutant_num==0].groupby("exp").apply(lambda x: x.loc[x.mu < x.mu.quantile(.05)])
EXP_REP_null = oligos_by_exp.loc[lambda x:x.mutant_num==0].groupby("exp").apply(lambda x: x.loc[x.mu < x.mu.quantile(.5)])

EXP_activator_oligos = EXP_REP_activators.groupby("exp_nm").apply(lambda g1:g1.loc[g1.oligo.isin(g1.set_index("rep").loc[1].oligo) & g1.oligo.isin(g1.set_index("rep").loc[2].oligo)])
EXP_repressor_oligos = EXP_REP_repressors.groupby("exp_nm").apply(lambda g1:g1.loc[g1.oligo.isin(g1.set_index("rep").loc[1].oligo) & g1.oligo.isin(g1.set_index("rep").loc[2].oligo)])
EXP_null_oligos = EXP_REP_null.groupby("exp_nm").apply(lambda g1:g1.loc[g1.oligo.isin(g1.set_index("rep").loc[1].oligo) & g1.oligo.isin(g1.set_index("rep").loc[2].oligo)])

EXP_activator_rep_rate =  EXP_activator_oligos.groupby(level="exp_nm").apply(lambda x:x.oligo.nunique()) / EXP_REP_activators.groupby("exp_nm").apply(lambda x:x.groupby("rep").oligo.nunique().mean())
EXP_repressor_rep_rate =  EXP_repressor_oligos.groupby(level="exp_nm").apply(lambda x:x.oligo.nunique()) / EXP_REP_repressors.groupby("exp_nm").apply(lambda x:x.groupby("rep").oligo.nunique().mean())
EXP_null_rep_rate =  EXP_null_oligos.groupby(level="exp_nm").apply(lambda x:x.oligo.nunique()) / EXP_REP_null.groupby("exp_nm").apply(lambda x:x.groupby("rep").oligo.nunique().mean())
#EXP_repressor_rep_rate =  EXP_repressor_oligos.groupby(level="exp_nm").reset_index("oligo").oligo.nunique().mean() / EXP_REP_repressors.groupby("exp_nm").reset_index("oligo").oligo.nunique()

DLD1_activator_rediscovery_rate = EXP_activator_rep_rate.loc["DLD1_WT"]
U2OS_activator_rediscovery_rate = EXP_activator_rep_rate.loc["U2OS_WT"]
MAX_activator_rediscovery_celltype = EXP_activator_rep_rate.idxmax()

pprint({
    "EXP_activators_mean_len":EXP_activator_oligos.groupby(level="exp_nm").oligo.nunique().mean(),
    "EXP_repressors_mean_len":EXP_repressor_oligos.groupby(level="exp_nm").oligo.nunique().mean(),
    "EXP_null_mean_len":EXP_null_oligos.groupby(level="exp_nm").oligo.nunique().mean(),
    "EXP_repressor__mean_rep_rate":EXP_repressor_rep_rate.mean(),
    "EXP_activator_mean_rep_rate":EXP_activator_rep_rate.mean(),
    "EXP_null_mean_rep_rate":EXP_null_rep_rate.mean(),
    "U2OS_activator_rediscovery_rate":U2OS_activator_rediscovery_rate,
    "DLD1_activator_rediscovery_rate":DLD1_activator_rediscovery_rate,
    "MAX_activator_rediscovery_celltype":MAX_activator_rediscovery_celltype,
    "EXP_activator_rep_rate": EXP_activator_rep_rate,
})


plt.gcf().set_size_inches(6,4)
ax = plt.gca()
plt.hist([(np.log2(g.mu) - np.log2(g.mu.mean())).rename(k) for k, g in oligos_by_exp.groupby("exp_nm")])

ax.set_title("histogram of normalized log2 expression rates by experiment")


Observing that APOBEC is expressed in DLD1 and U2OS cells, we defined a further set of activators defined as “APOBEC-on” enhancers, consisting of those enhancers which were identified both in the DLD1 replicates and the U2OS replicates, this set had {100} elements. Lastly, observing that APOBEC is not expressed in HCT116 cells, we defined a final set of activators of interest, consisting of {59} possible transcription activators which exhibited high transcription in U2OS and DLD1 cells (ie, in the APOBEC-on enhancers), but not in HCT116 cells. These were called “APOBEC-exclusive” candidate enhancer regions. Observing that U2OS expression tended to differ greatly with DLD1 expression, and U2OS expression was overall higher, we also investigated another set, "U2OS-on", with U2OS-identified enhancers, excluding those which were detected in HCT116 peaks.


In [ ]:

APOBEC_on_oligo_ids = set(EXP_activator_oligos.loc["DLD1_WT"].oligo).intersection(EXP_activator_oligos.loc["U2OS_WT"].oligo)
APOBEC_sometimes_oligo_ids = set(EXP_activator_oligos.loc["DLD1_WT"].oligo).union(EXP_activator_oligos.loc["U2OS_WT"].oligo)
ABon_ids = APOBEC_on_oligo_ids
ABst_ids = APOBEC_sometimes_oligo_ids


APOBEC_exclusive_oligo_ids =APOBEC_on_oligo_ids.difference(EXP_activator_oligos.loc["HCT116_WT"].oligo)
U2OS_on_oligo_ids = set(EXP_activator_oligos.loc["U2OS_WT"].oligo)


ABex_ids = APOBEC_exclusive_oligo_ids 
U2on_ids = U2OS_on_oligo_ids
U2ex_ids = set(U2OS_on_oligo_ids).difference(EXP_activator_oligos.loc["HCT116_WT"].oligo)

ALLen_ids = set(EXP_activator_oligos.oligo)
WTst_ids =APOBEC_sometimes_oligo_ids.union(EXP_activator_oligos.loc["HCT116_WT"].oligo)


AB_differential_enhancer_ids = obe.loc[lambda x: x.oligo.isin(ABst_ids)].\
        loc[lambda df:abs(df.U2OS_WT_rank -df.DLD1_WT_rank) > 100].oligo.unique()

In [ ]:

oligo_exp_activator_matrix = pd.Series(1, index =EXP_activator_oligos.oligo.to_frame().reset_index().set_index(["exp_nm","oligo"]).index).loc[lambda df: ~df.index.duplicated(keep='first') ].unstack("exp_nm").fillna(0)
oligo_exp_activator_matrix = oligo_exp_activator_matrix.append( [ pd.Series(0, index = oligo_exp_activator_matrix.columns).rename(o) for o in [e for e in obe.oligo.unique() if e not in oligo_exp_activator_matrix.index] ])
correlations = oligo_exp_activator_matrix.corr()
for nm in correlations.columns: correlations.loc[nm,nm] = 0

intersection_grid = np.array([[np.sum(oligo_exp_activator_matrix[c1].values * oligo_exp_activator_matrix[c2].values)
     for c1 in oligo_exp_activator_matrix.columns ]
     for c2 in oligo_exp_activator_matrix.columns])

sns.clustermap((oligo_exp_activator_matrix.loc[oligo_exp_activator_matrix.sum(axis=1) >0].filter(regex=".*WT.*" )),figsize=(3,3))
#sns.clustermap(oligo_exp_activator_matrix.filter(regex=".*WT.*" ).T.filter(regex=".*WT.*" ),figsize=(3,3))
sns.clustermap((oligo_exp_activator_matrix.loc[oligo_exp_activator_matrix.sum(axis=1) >0]),figsize=(3,3))

f,subs = plt.subplots(1,2)
f.set_size_inches(9,5)

plt.sca(subs[0])
sns.lineplot(x = "U2OS_WT_rank", y = "mu",  hue = "exp_nm", data = oligos_by_exp.loc[lambda x: x.oligo.isin(EXP_activator_oligos.loc["U2OS_WT"].oligo)])
plt.legend(loc=[0,1.1])

plt.sca(subs[1])
sns.lineplot(x = "DLD1_WT_rank", y = "mu",  hue = "exp_nm", data = oligos_by_exp.loc[lambda x: x.oligo.isin(EXP_activator_oligos.loc["DLD1_WT"].oligo)],legend=False)


#### Do enhancer regions exhibit differential activity in the cell types under study? ####
We had previously identified separate groups of top5% enhancers in each cell type under study. Furthermore, we observed that the groups of top5% enhancers were different for each of the cell types. To define APOBEC-on activators, we took an intersection of the top5% enhancers. To study differential expression in these cell types, we defined a new set of APOBEC-sometimes enhancer regions consisting of the union of top5% enhancer oligos in each of these (ABst) experiments. 



There were {100} total enhancer locations in the ABst set. For each enhancer, we examined the descending rank-sorted order of that enhancer in the experimental data for that locus, assigning a number equal to the absolute value of that rank sort-order difference in each set to each of the APOBEC-somstetimes enhancer regions. Looking at the full set of APOBEC-sometimes enhancer regions, we found a mean ranksort discrepancy of {248} between the two data sets. To quantify significance of this number, we also computed the mean ranksort discrepancy of the APOBEC-sometimes enhancers in each of the two experimental replicates of DLD1 and U2OS. We found that the average absolute ranksort differential of APOBEC-sometimes enhancer candidates was [111] and [49] in the DLD1 and U2OS replicates respectively, with an overall mean of [250]. Importantly, we found that amongst the U2OSon and DLD1on oligos, the average rank changes between replicates was 7.63 and 12.75 respectively, leading us to conclude that for the most expressed oligos, the expression rank change provided an internally consistent number.

Equipped with this measure, we set a threshold of rank expression change of 100 ranks and observed that 47 of the ABst potential enahncers changed in rank by 100 or more between the two experiments. We called these “APOBEC-differential-enhancers”, and noted that these 47 oligos were candidates for cell-type specific differential activation of APOBEC.


In [ ]:
abst_U2OSNKFB_U2OS_WT_rankdiff = obe.loc[lambda x: x.oligo.isin(ABst_ids)].\
        apply(lambda df:abs(df.U2OS_WT_rank -df.U2OS_NFKB_rank),axis=1)

all_U2OSNKFB_U2OS_WT_rankdiff = obe.loc[lambda x:  x.mutant_num == 0].\
        apply(lambda df:abs(df.U2OS_WT_rank -df.U2OS_NFKB_rank),axis=1)

abst_dld1_U2OS_WT_rankdiff = obe.loc[lambda x: x.oligo.isin(ABst_ids)].\
        apply(lambda df:abs(df.U2OS_WT_rank -df.DLD1_WT_rank),axis=1)

all_dld1_U2OS_WT_rankdiff = obe.loc[lambda x: x.mutant_num == 0].\
        apply(lambda df:abs(df.U2OS_WT_rank -df.DLD1_WT_rank),axis=1)

AB_differential_enhancer_ids = obe.loc[lambda x: x.oligo.isin(ABst_ids)].\
        loc[lambda df:abs(df.U2OS_WT_rank -df.DLD1_WT_rank) > 100].oligo.unique()

print(f""" mean difference between rank sorts in U2OS vs DLD1 cells over--
   ABST oligos: ({abst_dld1_U2OS_WT_rankdiff.mean():.2f})  
   ALL wt oligos: ({all_dld1_U2OS_WT_rankdiff.mean():.2f})

Looking at only U2OS cell lines in NFKB-ko and NFKB designs, we find that the differences are reduced
   ABST oligos: ({abst_U2OSNKFB_U2OS_WT_rankdiff.mean():.2f})  
   ALL oligos: ({all_U2OSNKFB_U2OS_WT_rankdiff.mean():.2f})  


... and furthermore, that the rank change difference of expression levels amongst ABst expressed oligos is
reduced compared to the rank change difference between all oligos. In order to quantify the base level of
noise in rank fold change amongst all oligos, we looked at the average fold change difference between replicates,
finding that the overall change in expression ranks for 

ALL OLIGOS (U2OS) {obe_w_repchanges["U2OS_WT_rank_repchange"].abs().mean():.2f}
ABST OLIGOS (U2OS) {obe_w_repchanges.loc[lambda x: x.oligo.isin(ABst_ids)]["U2OS_WT_rank_repchange"].abs().mean():.2f}
ABST OLIGOS (U2OS) {obe_w_repchanges.loc[lambda x: x.oligo.isin(U2on_ids)]["U2OS_WT_rank_repchange"].abs().mean():.2f}


ALL OLIGOS (DLD1) {obe_w_repchanges["DLD1_WT_rank_repchange"].abs().mean():.2f}
ABST OLIGOS (DLD1) {obe_w_repchanges.loc[lambda x: x.oligo.isin(ABst_ids)]["DLD1_WT_rank_repchange"].abs().mean():.2f}
ABST OLIGOS (DLD1) {obe_w_repchanges.loc[lambda x: x.oligo.isin(EXP_activator_oligos.loc["DLD1_WT"].oligo)]["DLD1_WT_rank_repchange"].abs().mean():.2f}

Number of "APOBEC-differentialy-expressed" oligos:
    {len(AB_differential_enhancer_ids)} 

""")






Starting from this measure of differential expression in wild-type cells that were highly expressive of APOBEC, we sought to apply the same measure to quantification of differential expression between the APOBEC-expressing cell lines and the HCT116 cell type which does not natively express APOBEC. Defining an inclusive set of enhancers, WILDTYPE-activators with [XXX] members, consisting of the top5% enhancers from DLD1, HCT116, and U2OS, we looked at differential expression, computed between all 3x3 cell types for each of these enhancers. We were also curious to see how differential expression of the bot5% would vary between these three cell types and therefore defined the set WILDTYPE-repressors and computed the same score, using all 3x3 comparisons.

Using the same definitions above, we computed average and std deviation of ranksort discrepancies for activators ( [XXX]mean, [XXX]stddev ) and repressors  ( [XXX]mean, [XXX]stddev )  individually and identified as differentially expressed all those oligos which crossed the threshold of mean + 1 std discrepancy change. By this measure, we found that [XXX] / ([XXX]) oligos were differentially expressed between either DLD1 / (U2OS) and HCT116 cells respectively. We called their intersection, the “APOBEC-on-enhancers” (XXX regions). Comparing the set of oligos previously identified as in the DLD1 and U2OS sets but not in HCT116 cells (“ABOPEC-exclusive” oligos), we found that [XXX]% of APOBEC-exclusive oligos were found in this set of differentially expressed oligos, confirming the internal consistency of these two measures. 

In [ ]:

obe_wtst = obe.loc[lambda x: x.oligo.isin(WTst_ids) ]
celltypes = pd.Series(["DLD1_WT","U2OS_WT","HCT116_WT"], index = ["DLD1_WT","U2OS_WT","HCT116_WT"])
exptypes = pd.Series(obe.exp_type.unique(), index = obe.exp_type.unique())

enhancer_intersection_sizes = celltypes.apply(lambda x: celltypes.apply(lambda y:len(
    set(EXP_activator_oligos.loc[x].oligo.unique()).intersection(
    set(EXP_activator_oligos.loc[y].oligo.unique())))))

WTst_differential_expression_count = \
exptypes.apply(lambda ct1: 
        exptypes.apply(lambda ct2:
            len(set(obe_wtst.loc[lambda x:(abs(x[ct1 + "_rank"]  - x[ct2+"_rank"])) > 100].oligo.unique()))))





print(f"""
number of oligos highly expressed in at least one of: DLD1, U2OS, HCT116 WILDTYPE CELLS: {len(WTst_ids)}

Looking at the number of differentially expressed transcripts by delta-rank > 100 metric in each cell type, we
get:
   U2OS --> DLD1 {WTst_differential_expression_count.loc["U2OS_WT","DLD1_WT"]}
   U2OS --> HTC116 {WTst_differential_expression_count.loc["U2OS_WT","HCT116_WT"]}
   DLD1 --> HCT116 {WTst_differential_expression_count.loc["DLD1_WT","HCT116_WT"]}
   
Counts of differentially expressed transcripts in each cell type.

Looking at experiment type changes, we find 
   U2OS_WT --> U2OS_NFKB_KO {WTst_differential_expression_count.loc["U2OS_WT","U2OS_NFKB"]}
   HCT116_WT --> HCT116_GEM {WTst_differential_expression_count.loc["HCT116_WT","HCT116_GEM"]}
      """)


### Do enhancer regions exhibit differential activity in the cell types under study? ###
We had previously identified separate groups of top5% enhancers in each cell type under study. Furthermore, we observed that the groups of top5% enhancers were different for each of the cell types. To define APOBEC-on activators, we took an intersection of the top5% enhancers. To study differential expression in these cell types, we defined a new set of APOBEC-sometimes enhancer regions consisting of the union of top5% enhancer oligos in each of these experiments. 

There were [XXX] total enhancer locations in this set. For each enhancer, we examined the descending rank-sorted order of that enhancer in the experimental data for that locus, assigning a number equal to the absolute value of that rank sort-order difference in each set to each of the APOBEC-somstetimes enhancer regions. Looking at the full set of APOBEC-sometimes enhancer regions, we found a mean ranksort discrepancy of [XXX] between the two data sets. To quantify significance of this number, we also computed the mean ranksort discrepancy of the APOBEC-sometimes enhancers in each of the two experimental replicates of DLD1 and U2OS. We found that the average absolute ranksort differential of APOBEC-sometimes enhancer candidates was [XXX] and [XXX] in the DLD1 and U2OS replicates respectively, with an overall mean of [XXX] and overall standard deviation of [XXX]. We set a threshold of [XXX], which was the mean discrepancy + 1 STD as the ranksort discrepancy to define a given enhancer oligo as differentially activating between the two cell types and found that amongst the [XXX] oligos in APOBEC-sometimes, [XXX] ([XXX]%) “APOBEC-differential-activators”, were differentially expressed by this measure.

Starting from this measure of differential expression in wild-type cells that were highly expressive of APOBEC, we sought to apply the same measure to quantification of differential expression between the APOBEC-expressing cell lines and the HCT116 cell type which does not natively express APOBEC. Defining an inclusive set of enhancers, WILDTYPE-activators with [XXX] members, consisting of the top5% enhancers from DLD1, HCT116, and U2OS, we looked at differential expression, computed between all 3x3 cell types for each of these enhancers. We were also curious to see how differential expression of the bot5% would vary between these three cell types and therefore defined the set WILDTYPE-repressors and computed the same score, using all 3x3 comparisons.

Using the same definitions above, we computed average and std deviation of ranksort discrepancies for activators ( [XXX]mean, [XXX]stddev ) and repressors  ( [XXX]mean, [XXX]stddev )  individually and identified as differentially expressed all those oligos which crossed the threshold of mean + 1 std discrepancy change. By this measure, we found that [XXX] / ([XXX]) oligos were differentially expressed between either DLD1 / (U2OS) and HCT116 cells respectively. We called their intersection, the “APOBEC-on-enhancers” (XXX regions). Comparing the set of oligos previously identified as in the DLD1 and U2OS sets but not in HCT116 cells (“ABOPEC-exclusive” oligos), we found that [XXX]% of APOBEC-exclusive oligos were found in this set of differentially expressed oligos, confirming the internal consistency of these two measures. 




In [ ]:
diffex = exptypes.apply(lambda ct1: 
        exptypes.apply(lambda ct2:
            obe_wtst.loc[lambda x:(x[ct1 + "_rank"]  - x[ct2+"_rank"]) > 100].oligo.unique())).stack()\
    .groupby(level=[0,1]).apply(lambda x:pd.Series([e for i,e in enumerate(x.values[0])]))

diffex = diffex.rename("oligo").reset_index(level=2,drop=True)
diffex = diffex.rename_axis(["expressed","unexpressed"]).reset_index()


# ZERO AREA FILTERING #

In [ ]:
print("HI")

In [ ]:
all_obe = oligos_by_exp.reset_index().loc[lambda x:x.mutant_num<5].groupby(["starts","mutant_num","exp_type"]).mu.mean()

In [ ]:
filters = pd.concat(
    [all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([(df.values[ofs:ofs+4,:] * (
        np.concatenate([
            (np.zeros((4,1))+1),
            np.fliplr(1 - np.diag([1,1,1,1]))],
        axis=1))
               ).sum()  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("allothers") / 16,
     all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([(df.values[ofs:ofs+4,:] * (
        np.concatenate([
            (np.zeros((4,1))),
            np.fliplr( np.diag([1,1,1,1]))],
        axis=1))
               ).sum()  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("onlyablations") / 4,
     
     all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([
        (df.values[ofs:ofs+5,:][np.nonzero(
                       np.array([[0,0,0,0,1],
                       [0,0,0,1,1],
                       [0,0,1,1,0],
                       [0,1,1,0,0],
                       [0,1,0,0,0]]))]
               ).sum()   
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("allothers2") / 17,
     all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([np.std(df.values[ofs:ofs+4,:][np.nonzero (
        np.concatenate([
            (np.zeros((4,1))+1),
            np.fliplr(1 - np.diag([1,1,1,1]))],
        axis=1))
                                             ])
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("allothersstd"),
     all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([np.std(df.values[ofs:ofs+4,:])  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("std"),
         all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([
        (df.values[ofs:ofs+5,:][np.nonzero(
                       np.array([[0,0,0,0,1],
                       [0,0,0,1,1],
                       [0,0,1,1,0],
                       [0,1,1,0,0],
                       [0,1,0,0,0]]))]
               ).sum()  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("onlyablations2") / 8,
          all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([(df.values[ofs:ofs+4,:] * (
        np.concatenate([
            (np.zeros((4,1))),
            np.fliplr(1 - np.diag([1,1,1,1]))],
        axis=1))
               ).sum()  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("othermutants") / 12,
     
          all_obe.unstack(level=1).groupby(level=1).apply(
    lambda df:
    pd.Series([(df.values[ofs:ofs+4,:] * (
        np.concatenate([
            (np.zeros((4,1))+1),
            np.fliplr(0* np.diag([1,1,1,1]))],
        axis=1))
               ).sum()  
        for ofs in range(len(df.values) - 4)],index = df.index[4:])
).rename("onlywildtype") / 4,
     all_obe.unstack(level=1).groupby(level=1).apply(lambda df:
                                                
       pd.Series([ks_2samp( df.values[ofs:ofs+4,:][np.nonzero(   np.concatenate([
            (np.zeros((4,1))+1),
            np.fliplr(1-np.diag([1,1,1,1]))],
        axis=1))] ,
                 df.values[ofs:ofs+4,:][np.nonzero(  np.concatenate([
            (np.zeros((4,1))+0),
            np.fliplr(np.diag([1,1,1,1]))],
        axis=1))])[1]
          for ofs in range(len(df.values) - 4)],index = df.index[4:])).rename("ks_pval"),
     
          all_obe.unstack(level=1).groupby(level=1).apply(lambda df:
                                                
       pd.Series([ks_2samp( df.values[ofs:ofs+5,:][np.nonzero(
                       np.array([[0,0,0,0,1],
                       [0,0,0,1,1],
                       [0,0,1,1,0],
                       [0,1,1,0,0],
                       [0,1,0,0,0]]))],
           df.values[ofs:ofs+5,:][np.nonzero(
                       1- np.array([[0,0,0,0,1],
                       [0,0,0,1,1],
                       [0,0,1,1,0],
                       [0,1,1,0,0],
                       [0,1,0,0,0]]))]
       )[1]
          for ofs in range(len(df.values) - 4)],index = df.index[4:])).rename("ks2_pval"),
     
     
     all_obe.unstack(level=1).groupby(level=1).apply(lambda df:
                                                
       pd.Series([ks_2samp( df.values[ofs:ofs+4,:][np.nonzero(   np.concatenate([
            (np.zeros((4,1))+1),
            np.fliplr(1-np.diag([1,1,1,1]))],
        axis=1))] ,
                 df.values[ofs:ofs+4,:][np.nonzero(  np.concatenate([
            (np.zeros((4,1))+0),
            np.fliplr(np.diag([1,1,1,1]))],
        axis=1))])[0]
          for ofs in range(len(df.values) - 4)],index = df.index[4:])).rename("ks_stat")
    ],axis = 1).reset_index(level=2, drop=True)

In [ ]:
filters.index.names = ["exp_type","starts"]
filters["actual_starts"] = filters.index.get_level_values("starts") + 30
filters = filters.reset_index(level=1, drop = True).set_index("actual_starts",append=True)
filters.index.names = ["exp_type","mutant_start_position"]

In [ ]:
filters["mutdiff"] = filters["othermutants"] - filters["onlyablations"]
filters["wtdiff"] = filters["onlywildtype"] - filters["onlyablations"]\

filters["othersdiff"] = filters["allothers"] - filters["onlyablations"]
filters["rank_mutdiff"] = filters[["mutdiff"]].join(
        filters[["mutdiff"]].groupby("mutant_start_position").mean().reset_index().sort_values("mutdiff",ascending = False).reset_index().rename_axis("rank",axis="index").reset_index().set_index(["mutant_start_position"])["rank"],
    on="mutant_start_position")["rank"]
filters["rank_ao"] = filters[["othersdiff"]].join(
        filters[["othersdiff"]].groupby("mutant_start_position").mean().reset_index().sort_values("othersdiff",ascending = False).reset_index().rename_axis("rank",axis="index").reset_index().set_index(["mutant_start_position"])["rank"],
    on="mutant_start_position")["rank"]
filters["rank_ao_dld1"] = filters[["othersdiff"]].loc[lambda x: x.index.get_level_values(0).str.contains("DLD")].join(
        filters[["othersdiff"]].groupby("mutant_start_position").mean().reset_index().sort_values("othersdiff",ascending = False).reset_index().rename_axis("rank",axis="index").reset_index().set_index(["mutant_start_position"])["rank"],
    on="mutant_start_position")["rank"]
filters["rank_ao_u2os"] = filters[["othersdiff"]].loc[lambda x: x.index.get_level_values(0).str.contains("U2OS")].join(
        filters[["othersdiff"]].groupby("mutant_start_position").mean().reset_index().sort_values("othersdiff",ascending = False).reset_index().rename_axis("rank",axis="index").reset_index().set_index(["mutant_start_position"])["rank"],
    on="mutant_start_position")["rank"]
filters["rank_ao_hct116"] = filters[["othersdiff"]].loc[lambda x: x.index.get_level_values(0).str.contains("HCT")].join(
        filters[["othersdiff"]].groupby("mutant_start_position").mean().reset_index().sort_values("othersdiff",ascending = False).reset_index().rename_axis("rank",axis="index").reset_index().set_index(["mutant_start_position"])["rank"],
    on="mutant_start_position")["rank"]





In [ ]:
filters["filterchange"] = np.max(
    [(filters.onlyablations - filters.allothers),
     (filters.onlyablations2 - filters.allothers2)]
)
filters["ks_1or2"] = filters.apply(lambda x: (x.ks_pval < .05) | (x.ks2_pval < .05),axis =1)

In [ ]:
filters["filter_color"] =filters.apply( lambda x: 
                                       5 if (x.ks_1or2 & x.)

In [ ]:
filters["log_ks_pval"] = np.log(filters.ks_pval)

In [ ]:
print(len(filters.loc[filters.ks_pval < .05]), len(filters.loc[filters.ks2_pval < .05]))

In [ ]:
filters_hq = filters.loc[lambda x: x.ks_1or2]

In [ ]:
sns.scatterplot(x = "onlyablations",
                y=filters.apply(lambda x:x.allothers- x.onlyablations,axis =1),size = "allothersstd", 
hue = filters.ks_pval.apply(lambda x: 1 if x < .05 else 0),data = filters)



In [ ]:
sns.scatterplot

In [ ]:

sns.jointplot(x = "onlyablations",
                y="allothers",
                data = filters_hq.loc[lambda x:(x.ks_pval<.05) & (x.onlywildtype > 2.5)])

In [ ]:
fsub = filters_hq.loc[lambda x:(x.ks_pval<.05) & (x.onlywildtype > 2.5)]

In [ ]:
sns.jointplot(x ="log_ks_pval", y="allothers" , data = fsub)

In [ ]:
for ranking in ["rank_ao_u2os","rank_ao_dld1","rank_ao_hct116"]:
    f = plt.figure()
    f.set_size_inches(5,3)
    mubar_wt_cutoffs = filters.groupby("exp_type").onlywildtype.quantile(.95)
    sns.lineplot(x=ranking, y="mubar",
                 style="filter_name", 
                 hue="exp_type",
                 estimator=None,
                 data = filters.loc[lambda x:(x[ranking]< 100) 
                                    & (x.apply(lambda y:y.onlywildtype>mubar_wt_cutoffs[y.name[0]],axis=1))]\
                 .reset_index()\
                     .melt(id_vars=["rank_mutdiff","rank_ao",ranking,"mutant_start_position","exp_type"],
                           value_vars=["onlyablations","allothers"],
                           value_name="mubar",
                           var_name="filter_name"))



In [ ]:
for ranking in ["rank_ao_u2os","rank_ao_dld1","rank_ao_hct116"]:
    f = plt.figure()
    f.set_size_inches(5,3)
    mubar_wt_cutoffs = filters_hq.groupby("exp_type").onlywildtype.quantile(.95)
    sns.lineplot(x=ranking, y="mubar",
                 style="filter_name", 
                 hue="exp_type",
                 estimator=None,
                 data = filters_hq.loc[lambda x:
                                       (x.apply(lambda y:y.onlywildtype>mubar_wt_cutoffs[y.name[0]],axis=1))]\
                 .reset_index()\
                     .melt(id_vars=["rank_mutdiff","rank_ao",ranking,"mutant_start_position","exp_type"],
                           value_vars=["onlyablations","allothers"],
                           value_name="mubar",
                           var_name="filter_name"))



In [ ]:
print("HI")

### WHICH 30BP REGIONS PASS THE ZERO AREA FILTER & HAVE WT EXPRESSION LEVELS FALLING INTO THE TOP 5%? ###

In [ ]:
qvals =  [.75,.85,.9,.95,.98]

In [ ]:
filters.groupby("exp_type").onlywildtype.quantile(wt_quantile)

In [ ]:
filters_hq.apply(lambda y:(1 if (y.onlywildtype > mubar_wt_cutoffs.loc[y.name[0]]) else 0),axis=1).sum()

In [ ]:
filters_hq.loc[["DLD1_WT"]].apply(lambda y:(1 if (y.onlywildtype > mubar_wt_cutoffs.loc[y.name[0]]) else 0),axis=1).sum()

In [ ]:
these_filters.apply(lambda y:[y.filterchange ,filter_change_cutoff.loc[y.name[0]]],axis=1)["HCT116_WT"]

In [ ]:
these_filters.apply(lambda y:(1 if (y.filterchange > filter_change_cutoff.loc[y.name[0]]) else 0),axis=1).groupby(level=0).sum()



In [ ]:
filter_change_cutoff 

In [ ]:
quantiles = pd.DataFrame()
quantile_counts = pd.DataFrame()



for quantile in qvals:
    
    these_filters = filters_hq.copy()
    wt_quantile = quantile
    change_quantile = quantile

    #USES ALL EXPRESSION VALUES
    mubar_wt_cutoffs = filters.groupby("exp_type").onlywildtype.quantile(wt_quantile)
    
    #filters_hq["filterchange"] = filters_hq.onlyablations - filters_hq.allothers
    #filter_change_cutoff = filters.filterchange.groupby("exp_type").quantile(change_quantile)
    these_filters["filterchange"] =  these_filters.apply(lambda x:
                                                       np.max(np.abs([x.onlyablations - x.allothers,
                                                               x.onlyablations2 - x.allothers2,])),axis=1)
    filter_change_cutoff = these_filters.filterchange.groupby("exp_type").quantile(change_quantile)

    
    
    #change_filter_value = these_filters.filterchange.quantile(change_quantile)
    #change_filter_value = 1
    #filter_change_cutoff = these_filters.groupby("exp_type").apply(lambda x:change_filter_value)
    
    print(f"""
    wt_cutoff quantile {quantile} == {mubar_wt_cutoffs}
    change_cutoff mean quantile {quantile} == {filter_change_cutoff}
    """)
    these_filters["filterchange"] =  these_filters.apply(lambda x:
                                                       np.max(np.abs([x.onlyablations - x.allothers,
                                                               x.onlyablations2 - x.allothers2,])),axis=1)
    
    these_filters["wt_filtered"] = these_filters.apply(lambda y:(1 if (y.onlywildtype > mubar_wt_cutoffs.loc[y.name[0]]) else 0),axis=1)
    these_filters["change_filtered"] = these_filters.apply(lambda y:(1 if (y.filterchange > filter_change_cutoff.loc[y.name[0]]) else 0),axis=1)
    these_filters["both_filtered"] = these_filters.wt_filtered * these_filters.change_filtered

    these_filters["filter_color"] = these_filters.apply(
        lambda x: (3 if x.both_filtered 
                   else(2 if x.change_filtered 
                        else (1 if x.wt_filtered 
                              else 0) )),axis=1)

    quantiles = quantiles.append(these_filters.assign(quantile = quantile))
    out = pd.Series([1,2,3]).apply(lambda n:(these_filters["filter_color"].loc[lambda x:x>0].unstack(level=0).fillna(0) == n).sum()).assign(quantile = quantile)
    quantile_counts =quantile_counts.append(out)


In [ ]:
f,subs = plt.subplots(1,quantiles["quantile"].nunique())


f.set_size_inches(10,12)
for i,q in enumerate(sorted(quantiles["quantile"].unique())):
    plt.sca(subs[i])
    data = quantiles.loc[lambda df:df["quantile"]==q]
    sns.heatmap( data = (data["filter_color"]*(data["ks_pval"]<.1)).loc[lambda x:x>0].unstack(level=0).fillna(0))

    #plt.sca(subs[i*2+1])
    #sns.heatmap(data["log_ks_pval"].loc[lambda x:x<-2].unstack(level=0).fillna(0))

plt.tight_layout()

In [ ]:
affine_scores = [1,2,3,5]

qjoin_afs = pd.concat([
 pd.merge(
    quantiles.reset_index(),
    data.dropna(subset =["target_mutant_start_position"]).loc[lambda x: x.affine_score > afs],
    right_on="target_mutant_start_position",
    left_on="mutant_start_position").assign(affine_score_threshold = afs)
    
    for afs in affine_scores],axis=0)
                                                                                     

In [ ]:
#TESTS THE ABOVE CODE FOR CORRECTNESS
interesting = qjoin_afs.loc[lambda df:(df.mutant_start_position == 285377) & (df.exp_type=="U2OS_WT") & (df.affine_score_threshold==3)].sort_values("score",ascending=False)
foc_position = 285377
oligos_overlap = obe.loc[obe.starts.isin(range(foc_position - 150,foc_position)) & (obe.exp_type=="U2OS_WT")]
mut_seq = chrseq[region_bounds[0]:region_bounds[1]][foc_position:foc_position+30]
sequences = pd.concat([oligos_overlap.oligo,oligos_overlap.Sequences.str.contains(mut_seq).rename("contains_mut"), oligos_overlap.Sequences.str.contains("GGGGAATTCC").rename("contains_motif")],axis=1)
np.log2(sequences.merge(interesting.loc[lambda x: x["quantile"]==.98].loc[lambda df: df.jaspar_id ==31], on ="oligo")[["contains_mut","mu","contains_motif","oligo","jaspar_id","is_ablation_mut"]].fillna(0).groupby("is_ablation_mut").mu.mean())

In [ ]:
motif_hits_of_interest = motif_oligos_data.loc[lambda x: x.affine_score > 0]

In [ ]:
motif_filter_color_counts = pd.DataFrame()
len(motif_hits_of_interest)
for q in quantiles["quantile"].unique():
    qsel = qjoin_afs.loc[lambda x: x["quantile"] == q] 
    motif_filter_color_counts = motif_filter_color_counts.append(
        pd.concat([
            qsel.groupby(['jaspar_id','exp_type',"affine_score_threshold"]).filter_color.value_counts().rename("fc_count"),
            
        ],axis = 1).join(motif_hits_of_interest.groupby(['jaspar_id']).size().rename("n_total_hits")).assign(quantile=q)
    )


In [ ]:
# idx = pd.IndexSlice
# motif_filter_color_counts.loc[idx[:, :,3,3], idx[:]]

In [ ]:
motif_fcc_selected =  \
    motif_filter_color_counts.reset_index().loc[lambda x: (x["quantile"] ==.9) & (x["affine_score_threshold"] ==2)]

In [ ]:
jaspar_overall_frac_interesting = (motif_fcc_selected.loc[lambda x:x.filter_color>0].groupby("jaspar_id").fc_count.sum()/\
    motif_fcc_selected.groupby("jaspar_id").fc_count.sum()).fillna(0).rename("fraction_interesting")

In [ ]:

jaspar_overall_rank = jaspar_overall_frac_interesting.reset_index().sort_values("fraction_interesting",ascending=False)\
    .reset_index().set_index("jaspar_id")["index"]

In [ ]:
motif_filter_color_counts = motif_filter_color_counts.join(jaspar_overall_rank,on="jaspar_id")

In [ ]:
frac_motif_enriched3 = motif_filter_color_counts.reset_index("filter_color").groupby(["jaspar_id","exp_type","affine_score_threshold","quantile"])\
    .apply(lambda x:  x.set_index("filter_color").loc[3].fc_count.sum()/x.set_index("filter_color").loc[:].fc_count.sum() if (3 in x.filter_color.values )  else 0 ).rename("color_3_enrichment")
frac_motif_enriched2 = motif_filter_color_counts.reset_index("filter_color").groupby(["jaspar_id","exp_type","affine_score_threshold","quantile"])\
    .apply(lambda x:  x.set_index("filter_color").loc[2].fc_count.sum()/x.set_index("filter_color").loc[:].fc_count.sum() if (2 in x.filter_color.values )   else 0 ).rename("color_2_enrichment")
frac_motif_enriched1 = motif_filter_color_counts.reset_index("filter_color").groupby(["jaspar_id","exp_type","affine_score_threshold","quantile"])\
    .apply(lambda x:  x.set_index("filter_color").loc[1].fc_count.sum()/x.set_index("filter_color").loc[:].fc_count.sum() if (1 in x.filter_color.values )  else 0 ).rename("color_1_enrichment")

In [ ]:


motif_filter_color_counts = motif_filter_color_counts.set_index("quantile",append=True).join(frac_motif_enriched1)\
    .join(frac_motif_enriched2)\
    .join(frac_motif_enriched3)



In [ ]:
plt.figure()
sns.lineplot(x="index", y = "color_2_enrichment", hue ="quantile", data = motif_filter_color_counts.reset_index())

color_enrichments = motif_filter_color_counts.reset_index().melt(
    id_vars=["jaspar_id","index","quantile"],
    value_vars = ["color_1_enrichment","color_2_enrichment","color_3_enrichment"],
    value_name = "enrichment_value",
    var_name ="enrichment_type",
)

plt.figure()
sns.lineplot(x = "index", y = "enrichment_value", hue = "enrichment_type", data = color_enrichments.loc[lambda x:x["quantile"]==.85])
             

plt.figure()
sns.lineplot(x="index", y = "color_3_enrichment", hue ="exp_type",
             data = motif_filter_color_counts.reset_index().loc[lambda x:x["quantile"] ==.75])

plt.figure()
sns.lineplot(x="index", y = "color_3_enrichment", hue ="exp_type",
             data = motif_filter_color_counts.reset_index().loc[lambda x:x["quantile"] ==.85])

plt.figure()
sns.lineplot(x="index", y = "color_1_enrichment", hue ="exp_type",
             data = motif_filter_color_counts.reset_index().loc[lambda x:x["quantile"] ==.95])

plt.figure()
sns.lineplot(x="index", y = "color_1_enrichment", hue ="exp_type",
             data = motif_filter_color_counts.reset_index().loc[lambda x:x["quantile"] ==.85])

In [ ]:
plt.imshow(motif_filter_color_counts.reset_index()\
    .loc[lambda x:(x["quantile"] ==.9) & (x["affine_score_threshold"] == 2) ]
    .groupby(["jaspar_id","exp_type"]).color_3_enrichment.max().unstack()>.0)

In [ ]:
motif_filter_color_counts.reset_index("affine_score_threshold").affine_score_threshold.unique()

In [ ]:
motif_filter_color_counts.loc[(idx[:],idx[:],idx[3]),:].color_3_enrichment.unique()

In [ ]:
thr = 2


    
plt.imshow(motif_filter_color_counts.reset_index()\
    .loc[lambda x:(x["quantile"] ==.9) & (x["affine_score_threshold"] == thr) ]
    .groupby(["jaspar_id","exp_type"]).color_3_enrichment.max().unstack()>0)

In [ ]:
thr = 2
color3 = motif_filter_color_counts.reset_index()\
    .loc[lambda x:(x["quantile"] ==.9) & (x["affine_score_threshold"] == thr) &  (x["color_3_enrichment"] >0 ) ]

U2_oligos =pd.concat( [
                       pd.Series("activator",index = EXP_activator_oligos.loc["U2OS_WT"].oligo),
pd.Series("repressor",index = EXP_repressor_oligos.loc["U2OS_WT"].oligo),
    pd.Series("null",index = EXP_null_oligos.loc["U2OS_WT"].oligo)
],axis = 0).rename("otype")





In [ ]:
oligo_types = U2_oligos.to_frame().set_index("otype",append=True).assign(val=1).reset_index().pivot_table(index="oligo",columns="otype")["val"].fillna(0)

In [ ]:
modata_thr = motif_oligos_data.loc[lambda x:(x.affine_score > thr) & (x.is_overlapping_wt)].set_index("jaspar_id").sort_index()
motif_wt_oligo_counts = jaspar.reset_index().apply(lambda x: len(modata_thr.loc[x.jaspar_id]) if x.jaspar_id in modata_thr.index.get_level_values(0) else 0,axis =1)


In [ ]:
oligo_type_counts =  modata_thr.groupby("jaspar_id").apply(lambda g1:
oligo_types.apply( 
    lambda y:
y.loc[[e for e in g1.oligo.unique() if e in y.index]].sum()
))

In [ ]:
oligo_type_fracs = oligo_type_counts.apply(lambda x: x / x.sum(), axis =1).fillna(0) / oligo_type_fracs.mean(axis=0)

In [ ]:
pd.concat(motif_filter_color_counts)

In [ ]:
color_counts_2d = (motif_filter_color_counts.reset_index()\
    .loc[lambda x:(x["quantile"] ==.95) & (x["affine_score_threshold"] == 1) ]\
    .groupby(["jaspar_id","exp_type"]).color_1_enrichment.max().unstack() ) * 1 
color_counts_2d = color_counts_2d /color_counts_2d.mean(axis=0)

In [ ]:

sns.heatmap(pd.concat([color_counts_2d, oligo_type_fracs[["activator","null","repressor"]]],axis = 1).fillna(0))

In [ ]:
plt.imshow(motif_filter_color_counts.reset_index()\
    .loc[lambda x:(x["quantile"] ==.95) & (x["affine_score_threshold"] == 1) ]
    .groupby(["jaspar_id","exp_type"]).color_1_enrichment.max().unstack().T>.0)

plt.figure()
plt.imshow(oligo_type_fracs[["activator","repressor"]].T>.1)

# MOTIF ANALYSIS #

### WHICH MOTIFS ARE FOUND IN REGIONS OF DIFFERENTIAL EXPRESSION BETWEEN WILD-TYPE CELLS? ###

### Which motifs are found in regions of differential expression between WT & perturbed cell types? ###

Do enhancer regions exhibit differential activity in chemical perturbations of the cell types under study? 

We examined the same discrepancy measure in the context of cellular perturbations of U2OS and HCT116 cells. 

First, considering U2OS cells with NFKB1 and NFKB2 knockouts, we created the differential expression sets in the same manner as defined above and found [XXX] enhancer regions to exhibit differential expression activation “NFKB-enhancers”, including [XXX] enhancers with ranksort values which changed by more than 3 STANDARD DEVIATIONS. Sorting these cells to look at only GFP negative cells, we found [XXX] enhancers with ranksort values which changed by 3 STANDARD DEVIATIONS.

Next, considering HCT116 cells subjected to treatment with 2uM gemtabicine, we found [XXX] enhancer regions exhibiting differential expression “GEMTAB-enhancers”. Amongst these, [XXX] of the enhancer regions had a ranksort difference of 3 STANDARD DEVIATIONS. 


In [ ]:
HCT116_perturbation_on =pd.Series(True, index =  diffex.loc[lambda x: (x.expressed=="HCT116_WT") & (x.unexpressed=="HCT116_GEM")].oligo).rename("HCT116_perturbed_off")
HCT116_perturbation_off =  pd.Series(True, index = diffex.loc[lambda x: (x.expressed=="HCT116_GEM") & (x.unexpressed=="HCT116_WT")].oligo).rename("HCT116_perturbed_on")
U2OS_knockout_on = pd.Series(True, index =   diffex.loc[lambda x: (x.expressed=="U2OS_WT") & (x.unexpressed=="U2OS_NFKB")].oligo).rename("U2OS_perturbed_off")
U2OS_knockout_off =  pd.Series(True, index= diffex.loc[lambda x: (x.expressed=="U2OS_NFKB") & (x.unexpressed=="U2OS_WT")].oligo).rename("USOS_perturbed_on")
perturbations = pd.concat([HCT116_perturbation_off,HCT116_perturbation_on,U2OS_knockout_off,U2OS_knockout_on],axis=1)
affine_score_threshold = 1
jaspar_perturbation_hits = perturbations.apply(lambda x: motif_oligos_data.loc[lambda x: x.affine_score >affine_score_threshold].groupby("jaspar_id").apply(lambda g:len(g.join(x.dropna(),on = "oligo",how ="inner").oligo.unique()))).reset_index()
jaspar_perturbation_fracs = perturbations.apply(lambda p: motif_oligos_data.loc[lambda df: df.affine_score >affine_score_threshold].groupby("jaspar_id").apply(lambda g:len(g.join(p.dropna(),on = "oligo",how ="inner").oligo.unique())/(g.oligo.nunique()*len(p.dropna())))).reset_index()
jaspar_perturbation_fracs_unstacked = jaspar_perturbation_fracs.melt(id_vars = "jaspar_id", value_name ="hit_frac",var_name="perturbation_type")
sns.scatterplot(x="jaspar_id",y="hit_frac", hue="perturbation_type", data = jaspar_perturbation_fracs_unstacked)

# MOTIF DISCOVERY #

### Which motifs do we discover in the unfiltered intervals ###

In [ ]:
import xml.etree.ElementTree as ET
root = ET.parse('/Users/ben/src/meme-5.0.5/meme_out_2200/meme.xml').getroot()
#arrays = [[[{e.attrib["letter_id"]: e.text} for e in v if e.attrib["letter_id"] in "ATGC"] for v in c[1].getchildren()[0].getchildren()] for c in root.getchildren()[2].getchildren()]
arrays = [[dict([[e.attrib["letter_id"],np.round( float(e.text) * 100)] for e in v if e.attrib["letter_id"] in "ATGC"]) for v in c[1].getchildren()[0].getchildren()] for c in root.getchildren()[2].getchildren()]

from Bio.motifs import jaspar
import io
import Bio
from Bio.Alphabet import IUPAC
motifs_string = "\n".join([">TEST{0} {0}\n".format(i) + "\n".join([k + " ["+" ".join([str(int(d[k])) for d in positions] ) +"]" for k in "ATGC"]) 
     for i, positions in enumerate(arrays)])
f = io.StringIO(motifs_string)
meme_motifs = jaspar.read(f,"jaspar")
intervals["bioseq"] = intervals.seq.apply(lambda x: Bio.Seq.Seq(x,alphabet = Bio.Alphabet.IUPAC.IUPACUnambiguousDNA()))

for mm in meme_motifs: mm.pseudocounts = 1

### Which motifs do we discover in the expressed oligos? ###

In [ ]:
#[TODO] run meme on the expressed oligos only, using the negative null set as the background control

### How do the set of discovered motifs compare to the reference set? ###

In [ ]:
jaspar = z2_save_jaspar.load_jaspar()

In [ ]:
memes =pd.Series(meme_motifs)

In [ ]:
memes = meme_motifs

In [ ]:
jaspar_array = np.array(list(e for e in jaspar.pwm.apply(lambda x: [e for k in "ATGC" for e in list(x[k])+([0]*(25 - len(x[k])))]).values))
memes_array = np.array(list(e for e in memes.apply(lambda x: [e for k in "ATGC" for e in list(x.pwm[k])+([0]*(25 - len(x.pwm[k])))]).values))
sns.clustermap(np.matmul( (jaspar_array / jaspar_array.sum(axis=1)[:,np.newaxis]),(memes_array/memes_array.sum(axis=1)[:,np.newaxis]).T))

In [ ]:
#LOADS BIOLOGICAL MOTIFS AND SCANS ALL SUBREGIONS FOR OCCURENCES
from pyfaidx import Fasta
sequences_fa = Fasta('/Users/ben/genomes/GRCh38.primary_assembly.genome.fa')
chrseq = str(sequences_fa["chr22"])
region_bounds=[ 38699734, 39291007]
background = dict([[l,chrseq[region_bounds[0]:region_bounds[1]].count(l) / len(chrseq[region_bounds[0]:region_bounds[1]])] for l in "ATGC"])


In [ ]:
meme_hits = memes.apply(lambda mm: intervals.apply(lambda x:[ e for e in  mm.pssm.search(x.bioseq)] , axis = 1))
meme_unstacked =pd.concat(
    [meme_hits.stack().groupby(level=[0,1]).apply(lambda g: pd.Series([e[0] for r in g for e in r])).rename("instance_idx"),
     meme_hits.stack().groupby(level=[0,1]).apply(lambda g: pd.Series([e[1] for r in g for e in r])).rename("instance_score")]
    ,axis = 1)

meme_unstacked.index.levels[0].name="meme_motif"
meme_unstacked.index.levels[1].name="interval_id"
meme_unstacked = meme_unstacked.reset_index(level=["meme_motif","interval_id"]).reset_index(drop = True)
meme_unstacked = meme_unstacked.join(memes.apply(lambda x:str(x.consensus)).rename("consensus"),on = "meme_motif")
meme_unstacked = meme_unstacked.join(intervals[["start","end","seq"]], on = "interval_id")
meme_unstacked["instance_position"] = meme_unstacked.apply(
    lambda x:x.start + x.instance_idx if x.instance_idx >= 0 else x.end + x.instance_idx,axis =1)
meme_unstacked["instance_seq"] = meme_unstacked.apply(lambda x:x.seq[x.instance_position - x.start:][:len(x.consensus)], axis = 1)

#LOADS BIOLOGICAL MOTIFS AND SCANS ALL SUBREGIONS FOR OCCURENCES
from pyfaidx import Fasta
sequences_fa = Fasta('/Users/ben/genomes/GRCh38.primary_assembly.genome.fa')
chrseq = str(sequences_fa["chr22"])
region_bounds=[ 38699734, 39291007]
background = dict([[l,chrseq[region_bounds[0]:region_bounds[1]].count(l) / len(chrseq[region_bounds[0]:region_bounds[1]])] for l in "ATGC"])

memes_patser = memes.apply(lambda x: x.pssm.distribution(background=background, precision=10**3).threshold_patser())
meme_unstacked = meme_unstacked.join(memes_patser.rename("motif_threshold_patser"),on="meme_motif")
meme_unstacked["affine_score"] = meme_unstacked["instance_score"] - meme_unstacked["motif_threshold_patser"]
meme_unstacked["instance_identity"] = meme_unstacked[["instance_seq","consensus"]].apply(lambda x: float(len([ i for i,l in enumerate(x.instance_seq) if l == x.consensus[i]]))/len(x.consensus),axis=1)


sns.scatterplot( x = "instance_identity", y = "affine_score", data= meme_unstacked)
plt.figure()
sns.scatterplot( x = "meme_motif", y = "affine_score", data= meme_unstacked)

# APPENDIX #
Additional figureds which may be useful in understanding the data

### APPENDIX A: RANKSORT DISCREPANCIES ###
The selection of APOBEC-differential-activators was performed based on a change in the ranksorted expression levels between the two APOBEC expressing cell types under study. For this to be a valid measure, we would hope that the majority of activators were discovered in both cell types and that their differences fell under the differential expression threshold. 

This is the case to a limited extent. 

In [ ]:

f,subs = plt.subplots(2,2)
f.set_size_inches(7,7)
plt.sca(subs[0][0])
sns.scatterplot(obe.U2OS_WT_rank,obe.DLD1_WT_rank)

plt.sca(subs[0][1])

sns.scatterplot(
    x = "U2OS_WT_rank",
    y = "DLD1_WT_rank",
    data =  obe.loc[lambda x: x.oligo.isin(ABon_ids)],
)
plt.gca().set_title(f"rank of {len(ABon_ids)} ABex oligo ranks")


plt.sca(subs[1][0])
sns.scatterplot(
    x = "U2OS_WT_rank",
    y = "DLD1_WT_rank",
    data =  obe.loc[lambda x: x.oligo.isin(U2on_ids)],
)
plt.gca().set_title(f"rank of {len(U2on_ids)} U2on oligo ranks")
plt.tight_layout()

plt.gca().set_xlim([0,200])
plt.gca().set_ylim([0,200])


plt.sca(subs[1][1])
sns.scatterplot(
    x = "U2OS_WT_rank",
    y = "U2OS_NFKB_rank",
    data =  obe.loc[lambda x: x.oligo.isin(U2on_ids)],
)
plt.gca().set_title(f"rank of {len(U2on_ids)} ABon oligo ranks")
plt.tight_layout()

plt.gca().set_xlim([0,200])
plt.gca().set_ylim([0,200])

f = plt.figure()
ax = plt.gca()
ax.set_title ("histogram of rank change differenes between DLD1 and U2OS\n expression levels for all oligos in APOBEC-sometimes")

plt.hist(  (obe.loc[lambda x:x.oligo.isin(ABst_ids)].DLD1_WT_rank - 
            obe.loc[lambda x:x.oligo.isin(ABst_ids)].U2OS_WT_rank ).abs() ,
        bins = np.arange(0,1000,50))

f = plt.figure()
ax = plt.gca()
ax.set_title ("histogram of rank change differenes between DLD1 and U2OS\n expression levels for all oligos")

plt.hist(  (obe.DLD1_WT_rank - obe.U2OS_WT_rank ).abs(),
         bins = np.arange(0,1000,50) )


# obe_win = obe.loc[lambda x: x.oligo.isin(U2on_ids)]
# from scipy import stats
# def r2(x, y):
#     return stats.pearsonr(x, y)[0] ** 2

# def pval(x, y):
#     return stats.pearsonr(x, y)[1] ** 2

# sns.jointplot("U2OS_WT_rank","DLD1_WT_rank", kind="reg", stat_func=r2,data = obe_win)
# #sns.regplot("U2OS_WT_rank","DLD1_WT_rank",data = obe_win)

In [ ]:
EXP_activator_oligos